In [ ]:
!pip install --upgrade pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
pylibcudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>

In [ ]:
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Chargement des données
historical_data_path = '/content/drive/My Drive/AirLiquideJL/donnees historiques/prix/hourly_day_ahead_prices_2017_2020.parquet'
historical_df = pd.read_parquet(historical_data_path)

# Convertion de l'index en datetime
historical_df.index = pd.to_datetime(historical_df.index)

# Normalisation des prix
scaler = MinMaxScaler()
historical_df['price'] = scaler.fit_transform(historical_df[['price']])

In [ ]:
# Création des fenêtres
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)
# Utilisation des 24 dernières heures pour prédire la suivante
window_size = 24
X, y = create_sequences(historical_df['price'].values, window_size)

# Division des données en ensembles d'entraînement et de test
split_index = int(len(X) * 0.8)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Format LSTM (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Modèle LSTM
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(window_size, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entraînement
history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - loss: 0.0053 - val_loss: 9.2330e-04
Epoch 2/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - loss: 9.7495e-04 - val_loss: 4.7527e-04
Epoch 3/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 5.5955e-04 - val_loss: 2.3562e-04
Epoch 4/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 3.6809e-04 - val_loss: 2.0149e-04
Epoch 5/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 3.0833e-04 - val_loss: 1.9053e-04
Epoch 6/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 3.0409e-04 - val_loss: 1.9775e-04
Epoch 7/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 2.8096e-04 - val_loss: 2.5506e-04
Epoch 8/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 2.5762e-04 - val_loss: 1.8530e-04
Epoch 9/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 2.5320e-04 - val_loss: 1.8719e-04
Epoch 10/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 2.6288e-04 - val_loss: 2.0162e-04
Epoch 11/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - l

In [ ]:
y_pred_test = model.predict(X_test)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_test_inv = scaler.inverse_transform(y_pred_test)
y_pred_test = np.clip(y_pred_test, 0, 1)

# Évaluation des performances
mae_lstm = mean_absolute_error(y_test_inv, y_pred_test_inv)
rmse_lstm = np.sqrt(mean_squared_error(y_test_inv, y_pred_test_inv))
print(f"MAE LSTM : {mae_lstm:.6f}, RMSE LSTM : {rmse_lstm:.6f}")

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
MAE LSTM : 0.007540, RMSE LSTM : 0.012793


In [ ]:
# Visualisation des résultats avec Plotly
df_plot = pd.DataFrame({
    'Vraies valeurs': y_test_inv.flatten(),
    'Prédictions LSTM': y_pred_test_inv.flatten()
}, index=historical_df.index[-len(y_test):])

fig = px.line(df_plot, title="Comparaison des prédictions LSTM vs Réalité")
fig.show()

# Application de la data augmentation

In [ ]:
import numpy as np

    # Ajoute un bruit gaussien à la série temporelle
def jittering(data, noise_level=0.01):
    return data + np.random.normal(0, noise_level, size=data.shape)

    # Multiplie la série temporelle par un facteur aléatoire
def scaling(data, scale_factor=0.1):
    factor = np.random.uniform(1 - scale_factor, 1 + scale_factor)
    return data * factor

    # Prend une sous-partie de la série temporelle
def window_slicing(data, slice_ratio=0.1):

    slice_size = int(len(data) * (1 - slice_ratio))
    start_idx = np.random.randint(0, len(data) - slice_size)
    return data[start_idx:start_idx + slice_size]


# Ajout de bruit


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

# Appliquer l'augmentation sur les séries
augmentation_method = jittering
X_train_augmented = np.array([augmentation_method(x) for x in X_train])

modele_data_augmented = Sequential([
    Input(shape=(window_size, 1)),
    LSTM(50, return_sequences=True),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compilation et entraînement
modele_data_augmented.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
modele_data_augmented.fit(X_train_augmented, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0396 - mae: 0.0396 - val_loss: 0.0195 - val_mae: 0.0195
Epoch 2/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.0206 - mae: 0.0206 - val_loss: 0.0110 - val_mae: 0.0110
Epoch 3/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.0152 - mae: 0.0152 - val_loss: 0.0105 - val_mae: 0.0105
Epoch 4/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0140 - mae: 0.0140 - val_loss: 0.0091 - val_mae: 0.0091
Epoch 5/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0133 - mae: 0.0133 - val_loss: 0.0098 - val_mae: 0.0098
Epoch 6/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.0131 - mae: 0.0131 - val_loss: 0.0100 - val_mae: 0.0100
Epoch 7/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.0130 - mae: 0.0130 - val_loss: 0.0097 - val_mae: 0.0097
Epoch 8/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0129 - mae: 0.0129 - val_loss: 0.0092 - val_mae: 0.0092
Epoch 9/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/st

In [ ]:
# Évaluer les performances
y_pred_augmented = modele_data_augmented.predict(X_test)
mae_augmented = mean_absolute_error(y_test, y_pred_augmented)
rmse_augmented = np.sqrt(mean_squared_error(y_test, y_pred_augmented))

print(f"MAE avec augmentation : {mae_augmented:.6f}, RMSE avec augmentation : {rmse_augmented:.6f}")

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
MAE avec augmentation : 0.009146, RMSE avec augmentation : 0.014623


# Scaling

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

# Appliquer l'augmentation
augmentation_method = scaling
X_train_augmented = np.array([augmentation_method(x) for x in X_train])

modele_data_augmented_scaling = Sequential([
    Input(shape=(window_size, 1)),
    LSTM(50, return_sequences=True),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1, activation='sigmoid')
])


In [ ]:
# Compilation et entraînement
modele_data_augmented_scaling.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
modele_data_augmented_scaling.fit(X_train_augmented, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0416 - mae: 0.0416 - val_loss: 0.0227 - val_mae: 0.0227
Epoch 2/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.0268 - mae: 0.0268 - val_loss: 0.0206 - val_mae: 0.0206
Epoch 3/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.0211 - mae: 0.0211 - val_loss: 0.0159 - val_mae: 0.0159
Epoch 4/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0198 - mae: 0.0198 - val_loss: 0.0144 - val_mae: 0.0144
Epoch 5/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 0.0195 - mae: 0.0195 - val_loss: 0.0130 - val_mae: 0.0130
Epoch 6/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0192 - mae: 0.0192 - val_loss: 0.0150 - val_mae: 0.0150
Epoch 7/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0190 - mae: 0.0190 - val_loss: 0.0146 - val_mae: 0.0146
Epoch 8/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.0190 - mae: 0.0190 - val_loss: 0.0144 - val_mae: 0.0144
Epoch 9/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/ste

In [ ]:
# Évaluer les performances
y_pred_augmented = modele_data_augmented_scaling.predict(X_test)
mae_augmented = mean_absolute_error(y_test, y_pred_augmented)
rmse_augmented = np.sqrt(mean_squared_error(y_test, y_pred_augmented))

print(f"MAE avec augmentation : {mae_augmented:.6f}, RMSE avec augmentation : {rmse_augmented:.6f}")

219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
MAE avec augmentation : 0.012992, RMSE avec augmentation : 0.018430


# time_warping

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

# Appliquer l'augmentation
augmentation_method = window_slicing
num_augmented = int(0.5 * len(X_train))
X_train_augmented = np.concatenate([X_train[:num_augmented], np.array([jittering(x) for x in X_train[:num_augmented]])])
y_train_augmented = np.concatenate([y_train[:num_augmented], y_train[:num_augmented]])


modele_data_augmented_window_slicing2 = Sequential([
    Input(shape=(window_size, 1)),
    LSTM(50, return_sequences=True),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])


In [ ]:
# Compilation et entraînement
modele_data_augmented_window_slicing2.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
modele_data_augmented_window_slicing2.fit(X_train_augmented, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0521 - mae: 0.0521 - val_loss: 0.0300 - val_mae: 0.0300
Epoch 2/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.0373 - mae: 0.0373 - val_loss: 0.0277 - val_mae: 0.0277
Epoch 3/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0355 - mae: 0.0355 - val_loss: 0.0287 - val_mae: 0.0287
Epoch 4/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.0345 - mae: 0.0345 - val_loss: 0.0162 - val_mae: 0.0162
Epoch 5/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0340 - mae: 0.0340 - val_loss: 0.0321 - val_mae: 0.0321
Epoch 6/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.0337 - mae: 0.0337 - val_loss: 0.0146 - val_mae: 0.0146
Epoch 7/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0335 - mae: 0.0335 - val_loss: 0.0165 - val_mae: 0.0165
Epoch 8/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.0339 - mae: 0.0339 - val_loss: 0.0162 - val_mae: 0.0162
Epoch 9/20
876/876 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step

In [ ]:
# Évaluer les performances
y_pred_augmented = modele_data_augmented_window_slicing2.predict(X_test)
mae_augmented = mean_absolute_error(y_test, y_pred_augmented)
rmse_augmented = np.sqrt(mean_squared_error(y_test, y_pred_augmented))

print(f"MAE avec augmentation : {mae_augmented:.6f}, RMSE avec augmentation : {rmse_augmented:.6f}")

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
MAE avec augmentation : 0.013754, RMSE avec augmentation : 0.019768


# Modèle avec les données synthétiques


In [ ]:
import pandas as pd
import os
from google.colab import drive

synthetic_data_path = "/content/drive/My Drive/AirLiquideJL/scenarios synthetiques/prix/"
files = [f for f in os.listdir(synthetic_data_path) if f.endswith(".parquet")]

# Charger et concaténer les fichiers Parquet
df_list = [pd.read_parquet(os.path.join(synthetic_data_path, file)) for file in files]
synthetic_df = pd.concat(df_list, ignore_index=True)

# Vérifier
print("Aperçu des données synthétiques :")
print(synthetic_df.head())

Aperçu des données synthétiques :
   price  dayofweek  hourofday
0  31.66          0          0
1  31.06          0          1
2  27.81          0          2
3  27.54          0          3
4  25.04          0          4


In [ ]:
# Prendre seulement une fraction des données pour éviter l'overflow
sample_fraction = 0.1  # Prend 10% des données pour réduire la taille
num_samples = int(len(synthetic_df) * sample_fraction)

synthetic_df = synthetic_df.iloc[:num_samples]

start_date = "2023-01-01 00:00:00"

synthetic_df = synthetic_df.copy()
synthetic_df["datetime"] = pd.date_range(start=start_date, periods=len(synthetic_df), freq="h")
synthetic_df.set_index("datetime", inplace=True)

In [ ]:
synthetic_df["computed_dayofweek"] = synthetic_df.index.dayofweek
synthetic_df["computed_hourofday"] = synthetic_df.index.hour

# Vérifier si la reconstruction est correcte
print(synthetic_df[["dayofweek", "computed_dayofweek", "hourofday", "computed_hourofday"]].head(10))

                     dayofweek  computed_dayofweek  hourofday  \
datetime                                                        
2023-01-01 00:00:00          0                   6          0   
2023-01-01 01:00:00          0                   6          1   
2023-01-01 02:00:00          0                   6          2   
2023-01-01 03:00:00          0                   6          3   
2023-01-01 04:00:00          0                   6          4   
2023-01-01 05:00:00          0                   6          5   
2023-01-01 06:00:00          0                   6          6   
2023-01-01 07:00:00          0                   6          7   
2023-01-01 08:00:00          0                   6          8   
2023-01-01 09:00:00          0                   6          9   

                     computed_hourofday  
datetime                                 
2023-01-01 00:00:00                   0  
2023-01-01 01:00:00                   1  
2023-01-01 02:00:00                   2  
2023-01-0

In [ ]:
print("Taille du DataFrame :", synthetic_df.shape)
print("Colonnes disponibles :", synthetic_df.columns)
print("Aperçu des 5 premières lignes :")
print(synthetic_df.head())


Taille du DataFrame : (538137, 5)
Colonnes disponibles : Index(['price', 'dayofweek', 'hourofday', 'computed_dayofweek',
       'computed_hourofday'],
      dtype='object')
Aperçu des 5 premières lignes :
                     price  dayofweek  hourofday  computed_dayofweek  \
datetime                                                               
2023-01-01 00:00:00  31.66          0          0                   6   
2023-01-01 01:00:00  31.06          0          1                   6   
2023-01-01 02:00:00  27.81          0          2                   6   
2023-01-01 03:00:00  27.54          0          3                   6   
2023-01-01 04:00:00  25.04          0          4                   6   

                     computed_hourofday  
datetime                                 
2023-01-01 00:00:00                   0  
2023-01-01 01:00:00                   1  
2023-01-01 02:00:00                   2  
2023-01-01 03:00:00                   3  
2023-01-01 04:00:00                   

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
synthetic_df["price"] = scaler.fit_transform(synthetic_df[["price"]])

def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

window_size = 24  # Utiliser 24 heures pour prédire la suivante
X_synthetic, y_synthetic = create_sequences(synthetic_df["price"].values, window_size)

# Reshape pour LSTM
X_synthetic = X_synthetic.reshape((X_synthetic.shape[0], X_synthetic.shape[1], 1))

print("Taille des données pour LSTM :", X_synthetic.shape)


Taille des données pour LSTM : (538113, 24, 1)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Séparer en train/test
split_index = int(len(X_synthetic) * 0.8)
X_train_syn, X_test_syn = X_synthetic[:split_index], X_synthetic[split_index:]
y_train_syn, y_test_syn = y_synthetic[:split_index], y_synthetic[split_index:]

# Définir le modèle LSTM
modele_synthetic = Sequential([
    LSTM(50, return_sequences=True, input_shape=(window_size, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1, activation='sigmoid')  # Assurer des valeurs entre [0,1]
])

modele_synthetic.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

modele_synthetic.fit(X_train_syn, y_train_syn, batch_size=32, epochs=20, validation_data=(X_test_syn, y_test_syn), callbacks=[early_stopping])

y_pred_syn = modele_synthetic.predict(X_test_syn)

# Évaluation des performances
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_syn = mean_absolute_error(y_test_syn, y_pred_syn)
rmse_syn = np.sqrt(mean_squared_error(y_test_syn, y_pred_syn))

print(f"MAE avec données synthétiques : {mae_syn:.6f}, RMSE : {rmse_syn:.6f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/20
13453/13453 ━━━━━━━━━━━━━━━━━━━━ 134s 10ms/step - loss: 0.0171 - mae: 0.0171 - val_loss: 0.0095 - val_mae: 0.0095
Epoch 2/20
13453/13453 ━━━━━━━━━━━━━━━━━━━━ 133s 9ms/step - loss: 0.0096 - mae: 0.0096 - val_loss: 0.0093 - val_mae: 0.0093
Epoch 3/20
13453/13453 ━━━━━━━━━━━━━━━━━━━━ 153s 10ms/step - loss: 0.0092 - mae: 0.0092 - val_loss: 0.0079 - val_mae: 0.0079
Epoch 4/20
13453/13453 ━━━━━━━━━━━━━━━━━━━━ 140s 10ms/step - loss: 0.0090 - mae: 0.0090 - val_loss: 0.0080 - val_mae: 0.0080
Epoch 5/20
13453/13453 ━━━━━━━━━━━━━━━━━━━━ 144s 10ms/step - loss: 0.0089 - mae: 0.0089 - val_loss: 0.0079 - val_mae: 0.0079
Epoch 6/20
13453/13453 ━━━━━━━━━━━━━━━━━━━━ 123s 9ms/step - loss: 0.0088 - mae: 0.0088 - val_loss: 0.0086 - val_mae: 0.0086
Epoch 7/20
13453/13453 ━━━━━━━━━━━━━━━━━━━━ 145s 9ms/step - loss: 0.0087 - mae: 0.0087 - val_loss: 0.0077 - val_mae: 0.0077
Epoch 8/20
13453/13453 ━━━━━━━━━━━━━━━━━━━━ 131s 10ms/step - loss: 0.0087 - mae: 0.0087 - val_loss: 0.0077 - val_mae: 0.0077
Epo

In [ ]:
# Évaluer les performances
y_pred_augmented = modele_synthetic.predict(X_test)
mae_augmented = mean_absolute_error(y_test, y_pred_augmented)
rmse_augmented = np.sqrt(mean_squared_error(y_test, y_pred_augmented))

print(f"MAE avec augmentation : {mae_augmented:.6f}, RMSE avec augmentation : {rmse_augmented:.6f}")

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
MAE avec augmentation : 0.006537, RMSE avec augmentation : 0.011289
